In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import keras
import os
import sys

sys.path.append('..')
import data
import model
import preprocessing
import utils

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.
C:\Users\PC\Anaconda3\envs\TensorFlow-GPU-Keras\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
params = utils.yaml_to_dict('config.yml')

## Primer paso: preprocesar los datos 
Hacerlo solo una vez

In [13]:
#preprocessing.download_data(params)
#preprocessing.extract_data(params)
#preprocessing.generate_spectogram_images(params)
#preprocessing.make_id_label_map(params)
preprocessing.split_data(params)

## Segund Paso: instanciar el modelo

In [3]:
inputs = tf.keras.layers.Input(shape=(224,224, 3))
net = model.ModelArchitecture(num_classes=params['num_classes'])
x = net(inputs, training=False)
net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 112, 112, 64)      9472      
_________________________________________________________________
batch_normalization (BatchNo (None, 112, 112, 64)      256       
_________________________________________________________________
activation (Activation)      (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 192)       110784    
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 192)       768       
_________________________________________________________________
activation_1 (Activation)    (None, 56, 56, 192)       0         
__________

## Tercer paso: crear el generador

In [4]:
print('Data in train')
train_generator = data.make_datagenerator(params, mode='training')
print('Data in validation')
val_generator = data.make_datagenerator(params,mode='validation')

Data in train
Found 1 images.
Data in validation
Found 1 images.


## Cuarto paso: correr el modelo

In [5]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(params['model_dir'], 'tf_ckpt'), 
    save_weights_only=True, 
    verbose=1,
    period=5)

tb_callback = tf.keras.callbacks.TensorBoard(
    os.path.join(params['model_dir'], 'logs'))

optimizer = tf.keras.optimizers.Adam()

steps_per_epoch = 50 #train_generator.n // params['batch_size']
validation_steps = 15 #val_generator.n // params['batch_size']

net.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
net.fit_generator(
    train_generator, 
    steps_per_epoch=steps_per_epoch, 
    epochs=params['num_epochs'],
    workers=4,
    validation_data=val_generator, 
    validation_steps=validation_steps,
    callbacks=[cp_callback, tb_callback])

Epoch 1/30
50/50 [==============================] - 8s 169ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 2/30
50/50 [==============================] - 7s 136ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 3/30
50/50 [==============================] - 7s 142ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 4/30
50/50 [==============================] - 7s 141ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 5/30
49/50 [============================>.] - ETA: 0s - loss: 1.1921e-07 - acc: 1.0000
Epoch 00005: saving model to checkpoints\tf_ckpt

Consider using a TensorFlow optimizer from `tf.train`.
50/50 [==============================] - 7s 145ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 6/30
50/50 [==============================] - 7s 137ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.